In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np 
import glob, os, subprocess
from dateutil import parser

import matplotlib.pyplot as plt
import matplotlib
import datacube
import sys, shutil
sys.path.append("..")

from SRC import *

%matplotlib inline
matplotlib.rcParams.update({'font.size': 11})
matplotlib.rcParams['savefig.dpi'] = 300


indir = '../MULLION/20MAR20/FLAME/SPECTRA/AVG/'
outdir = '../PNGS/MULLION/20MAR20/'
calfile = '../CALIBRATION/RadCalCoeff_GT08_.csv'
#TelemetryFile = '/g/data/u46/users/aw3463/GuyByrne/calval/FLAME/MULLION/20MAR20/TELEMETRY/Feb-20th-2020-12-46PM-Flight-Airdata.csv'

field_data = ['MUL', '20MAR20', '', 'Sentinel2a', 'GA 10inch Panel 99aa03-0319-1952', 'Radiance', '', '']

fignum=1

firstGoodLine = 1
firstGoodPanelSpec = 0
firstGoodGroundSpec = 132

brdf_data = np.array([['', 'brdf0', 'brdf1', 'brdf2'],
                          ['band1', 0.06538, 0.01958666666666667, 0.009953333333333333],
                          ['band11', 0.39993333333333336, 0.11646333333333335, 0.05016],
                          ['band12', 0.2479, 0.061623333333333336, 0.03579333333333334],
                          ['band2', 0.06538, 0.01958666666666667, 0.009953333333333333],
                          ['band3', 0.11231, 0.04193666666666667, 0.01812666666666667],
                          ['band4', 0.12977000000000002, 0.04653666666666667, 0.02082666666666667],
                          ['band5', 0.12977000000000002, 0.04653666666666667, 0.02082666666666667],
                          ['band6', 0.12977000000000002, 0.04653666666666667, 0.02082666666666667],
                          ['band7', 0.32748000000000005, 0.16837333333333335, 0.03345],
                          ['band8', 0.32748000000000005, 0.16837333333333335, 0.03345],
                          ['band8a', 0.32748000000000005, 0.16837333333333335, 0.03345],
                         ])

plot_scale = [0.0, 1.0, 0.0, 1.0]
Corners = [0, 0, 0, 0, 0, 0, 0, 0]
RockWalk = True
StartCorner = 'SE'
panel_dir =  '../../../CalVal_Phase1/RAW_DATA/Panels/'
in_panel = 'GA_10inch.txt'

sat_resp = dict({'Landsat5': '../../../CalVal_Phase1/SENSOR_BANDS/landsat5_vsir.flt', 
            'Landsat7': '../../../CalVal_Phase1/SENSOR_BANDS/landsat7_vsir.flt', 
            'Landsat8': '../../../CalVal_Phase1/SENSOR_BANDS/landsat8_vsir.flt',
            'Sentinel2a': '../../../CalVal_Phase1/SENSOR_BANDS/Sent2a.flt',
            'Sentinel2b': '../../../CalVal_Phase1/SENSOR_BANDS/Sent2b.flt'})

f_name = sat_resp[field_data[3]]

#
# Colours used for plotting multi-coloured Lines
#
colpac=['#770000', '#FF0000', '#FF7700', '#FFFF00', '#77FF00', '#00FF00', 
        '#00FF77', '#00FFFF', '#0077FF', '#0000FF', '#000077', '#FF00FF', '#777777', '#770077', '#777700']



In [2]:
#TeleDat = pd.read_csv(TelemetryFile)

TeleDat.drop(['height_above_ground_at_drone_location(meters)', 
              'ground_elevation_at_drone_location(meters)',
              'satellites',
              'gpslevel',
              'voltage(v)',
              'max_altitude(meters)',
              'max_ascent(meters)',
              'max_speed(m/s)',
              'max_distance(meters)',
              ' xSpeed(m/s)',
              ' ySpeed(m/s)',
              ' zSpeed(m/s)',
              'isPhoto',
              'isVideo',
              'rc_elevator',
              'rc_aileron',
              'rc_throttle',
              'rc_rudder',
              'battery_percent',
              'voltageCell1',
              'voltageCell2',
              'voltageCell3',
              'voltageCell4',
              'voltageCell5',
              'voltageCell6',
              'current(A)',
              'battery_temperature(c)',
              'altitude(meters)',
              'ascent(meters)',
              'flycStateRaw',
              'flycState'], axis=1, inplace=True)

TeleDat.rename(columns={'time(millisecond)': 'time_ms',
                        'datetime(utc)': 'datetime',
                        'height_above_takeoff(meters)': 'height_local',
                        'altitude_above_seaLevel(meters)': 'height_sea',
                        'distance(meters)': 'distance(m)',
                        ' compass_heading(degrees)': 'compass_heading',
                        ' pitch(degrees)': 'pitch',
                        ' roll(degrees)': 'roll',
                        'gimbal_heading(degrees)': 'gimbal_heading',
                        'gimbal_pitch(degrees)': 'gimbal_pitch',
                        'message': 'comment'
                       }, inplace=True
              )

TeleDat['date_saved'] = [parser.parse(TeleDat.datetime[i]) - timedelta(minutes=21) + timedelta(milliseconds=int(TeleDat.time_ms[i])) for i in range(len(TeleDat.datetime))]

TeleDat = TeleDat[TeleDat.latitude!=0]

TeleDat = TeleDat[TeleDat.date_saved.dt.year >= 2020]

TeleDat.date_saved.unique()

alldata['latitude'] = [TeleDat.latitude[int((i - TeleDat.date_saved).abs().argsort()[:1].values)] for i in alldata.date_saved]

In [3]:
alldata = LoadData.load_from_dir(indir, calfile)

In [4]:
#alldata[alldata.index == 349.176].plot('date_saved', 'radiance')

In [5]:
alldata['radiance'] = (alldata.DN-476)*alldata.CalData/alldata.IntTime
#alldata['radiance'] = alldata['radiance']/10000
alldata['Line'] = 1
alldata['Latitude'] = -35.123277
alldata['Longitude'] = 148.862692

In [ ]:
#
# Bin spectra into integer wavelength bins, assuming that the input data
# has 0.3nm channels.
#
def wavelength_bin(alldata):
    # Create avgdata DataFrame, based on alldata input DataFrame
    # Reset index so that Wavelength is not the index.
    avgdata = alldata.copy()
    avgdata.reset_index(inplace=True)
    
    ### Make a mean of three rows for each of DN, Calibration, Radiance
    ###avgdata['DNmean'] = avgdata.DN.rolling(3, center=True).mean()
    ###avgdata['Calmean'] = avgdata.CalData.rolling(3, center=True).mean()
    ###avgdata['Radmean'] = avgdata.radiance.rolling(3, center=True).mean()
    
    # Remove first and last wavelengths which have NaNs in them.
    avgdata = avgdata[avgdata.Wavelength != 349.176]
    avgdata = avgdata[avgdata.Wavelength != 910.925]
    
    # Create empty output DataFrame
    outputDF = pd.DataFrame()
    ttmm = pd.DataFrame()
    
    # Loop through each spectrum in the dataframe
    for j in avgdata.SpecNum.unique():
        tempdata = avgdata[avgdata.SpecNum==j].groupby(pd.cut(avgdata[avgdata.SpecNum==j]['Wavelength'], np.arange(349.5,910.5,1.0))).mean()
        tempdata['date_saved'] = avgdata[avgdata.SpecNum==j].date_saved.values[0]
        tempdata['filename'] = avgdata[avgdata.SpecNum==j].filename.values[0]
        
        tempdata.index.rename('plop', inplace=True)
        tempdata.reset_index(inplace=True)
        tempdata.drop(columns='plop', inplace=True)
        tempdata.Wavelength = tempdata.Wavelength.round(0).astype(int)        
        
        # When finished making an entire spectrum, add it to the output DataFrame
        # and print how many have been done.
        if outputDF.empty:
            outputDF = tempdata.copy()
        
        else:
            if j % 100 == 0:
                ttmm = pd.concat([ttmm, tempdata])
                outputDF = pd.concat([outputDF, ttmm])
                ttmm = pd.DataFrame()
            
            elif j == len(avgdata.SpecNum.unique())-1:
                ttmm = pd.concat([ttmm, tempdata])
                outputDF = pd.concat([outputDF, ttmm])
            
            elif ttmm.empty:
                ttmm = tempdata.copy()
            
            else:
                ttmm = pd.concat([ttmm, tempdata])
                
            
        print('Completed ', j+1, '/', len(avgdata.SpecNum.unique()), ' spectra', end='\r', flush=True)

    # output final DataFrame
    return outputDF

avgdata = wavelength_bin(alldata)

In [ ]:
cutavgdata = avgdata[np.logical_and(avgdata.date_saved > datetime(2020,3,20,0,23,30), avgdata.date_saved < datetime(2020,3,20,0,58,45))]

In [ ]:
panel_names, all_panels, all_grounds = ExtractPanelsGrounds.extract_panels_grounds(cutavgdata)

In [ ]:
all_panel_spec = MakeSpecDF.make_spec_df(all_panels)

In [ ]:
FIG_PanelRadiances.FIG_panel_radiances(all_panel_spec, outdir, field_data, fignum)

fignum += 1

In [ ]:
all_grounds_spec = MakeSpecDF.make_spec_df(all_grounds)

In [ ]:
FIG_GroundSpectra.FIG_ground_spectra(all_grounds_spec, outdir, field_data, fignum)
fignum +=1

In [ ]:
cutavgdata[abs(cutavgdata.Wavelength-450)<0.185].plot('date_saved', 'radiance')

In [ ]:
allASDdata = pd.read_pickle('../MULLION/20MAR20/ASD/alldat.pick')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12.5, 5.5))
plt.tight_layout(pad=5.5, w_pad=1.0, h_pad=1.0)
avgdata[abs(avgdata.Wavelength-550)<0.185].plot('date_saved', 'radiance', ax=axes)
allASDdata[allASDdata.Wavelength==550].plot('date_saved', 'radiance', ax=axes)
axes.set_ylim(0.0,0.53)
axes.set_xlim(datetime(2020,3,20,0,24), datetime(2020,3,20,0,58,45))
plt.legend(['Flame', 'ASD'])
plt.xlabel('UT Time (HH:MM)')
plt.ylabel('Radiance (W m$^{-2}$ nm$^{-1}$ sr$^{-1}$)')
plt.title('Mullion 20MAR20 joint ASD/Flame measurement using GT08 calibration')
axes.text(datetime(2020,3,20,0,25), 0.05, 'Ground')
axes.text(datetime(2020,3,20,0,28), 0.5, 'Panel')
axes.text(datetime(2020,3,20,0,31), 0.06, 'Ground')
axes.text(datetime(2020,3,20,0,35), 0.5, 'Panel')
axes.text(datetime(2020,3,20,0,38), 0.06, 'Ground')
axes.text(datetime(2020,3,20,0,41), 0.5, 'Panel')
axes.text(datetime(2020,3,20,0,43), 0.05, 'Ground')
axes.text(datetime(2020,3,20,0,46,30), 0.5, 'Panel')
axes.text(datetime(2020,3,20,0,49), 0.05, 'Ground')
axes.text(datetime(2020,3,20,0,52,30), 0.51, 'Panel')
axes.text(datetime(2020,3,20,0,55), 0.05, 'Ground')

axes.axvspan(datetime(2020,3,20,0,25,15), datetime(2020,3,20,0,26), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,31,38), datetime(2020,3,20,0,33,6), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,38,29), datetime(2020,3,20,0,39,4), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,44,9), datetime(2020,3,20,0,44,51), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,49,43), datetime(2020,3,20,0,50,40), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,55,47), datetime(2020,3,20,0,57,19), alpha=0.25, color='black', ec=None)

plt.savefig('plonk1.png')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12.5, 5.5))
plt.tight_layout(pad=5.5, w_pad=1.0, h_pad=1.0)
avgdata[abs(avgdata.Wavelength-550)<0.185].plot('date_saved', 'radiance', ax=axes)
allASDdata[allASDdata.Wavelength==550].plot('date_saved', 'radiance', ax=axes)
axes.set_ylim(0.02,0.06)
axes.set_xlim(datetime(2020,3,20,0,24), datetime(2020,3,20,0,58,45))
plt.legend(['Flame', 'ASD'])
plt.xlabel('UT Time (HH:MM)')
plt.ylabel('Radiance (W m$^{-2}$ nm$^{-1}$ sr$^{-1}$)')
plt.title('Mullion 20MAR20 joint ASD/Flame measurement using GT08 calibration')
axes.text(datetime(2020,3,20,0,25), 0.043, 'Ground')
axes.text(datetime(2020,3,20,0,28), 0.058, 'Panel')
axes.text(datetime(2020,3,20,0,32,30), 0.044, 'Ground')
axes.text(datetime(2020,3,20,0,35), 0.058, 'Panel')
axes.text(datetime(2020,3,20,0,38), 0.054, 'Ground')
axes.text(datetime(2020,3,20,0,41), 0.058, 'Panel')
axes.text(datetime(2020,3,20,0,43,30), 0.047, 'Ground')
axes.text(datetime(2020,3,20,0,46,30), 0.058, 'Panel')
axes.text(datetime(2020,3,20,0,49), 0.045, 'Ground')
axes.text(datetime(2020,3,20,0,52,30), 0.058, 'Panel')
axes.text(datetime(2020,3,20,0,55,30), 0.045, 'Ground')

axes.axvspan(datetime(2020,3,20,0,25,15), datetime(2020,3,20,0,26), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,31,38), datetime(2020,3,20,0,33,6), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,38,29), datetime(2020,3,20,0,39,4), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,44,9), datetime(2020,3,20,0,44,51), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,49,43), datetime(2020,3,20,0,50,40), alpha=0.25, color='black', ec=None)
axes.axvspan(datetime(2020,3,20,0,55,47), datetime(2020,3,20,0,57,19), alpha=0.25, color='black', ec=None)

plt.savefig('plonk2.png')

In [ ]:
allASDdata[allASDdata.Wavelength==450]

In [ ]:
avgdata.to_pickle('avgdata.pick')

In [ ]:
all_panels, all_grounds = SolarAngle.solar_angle(all_panels, all_grounds, field_data)

In [ ]:
all_panels = PanelBRF.Panel_BRF(all_panels, field_data)

In [ ]:
good_panels = all_panels.copy()
good_grounds = all_grounds.copy()

In [ ]:
gpta, adta = CreateTimeRelativeDF.create_time_relative_dfs(all_panels, all_grounds)

In [ ]:
FIG_AllTimelines.FIG_all_timelines(gpta, adta, outdir, field_data, fignum)
fignum += 1

In [ ]:
all_panel_mean = all_panel_spec.mean(axis=1)

gpta = FIG_NormalisedPanelsTimeline.normalise_spectra(all_panel_mean, all_panel_spec, gpta, field_data)

FIG_NormalisedPanelsTimeline.FIG_normalised_panels_timeline(gpta, outdir, field_data, fignum)

fignum += 1

In [ ]:
gpta['Latitude'] = -35.123277
gpta['Longitude'] = 148.862692
adta['Latitude'] = -35.123277
adta['Longitude'] = 148.862692

In [ ]:
slope, intercept, coszenith = FIG_InsolationFit.FIG_insolation_fit(gpta, outdir, field_data, fignum)
fignum += 1

### Scale good panels to insolation curve

Use the fitted insolation curve to determine individual offsets for each panel measurement and then divide<BR>
off the differences. This effectively removes any individual variations in the absolute scaling of panel<BR>
measurements.

In [ ]:
good_panels = ScalePanels.scale_panels(slope, intercept, coszenith, gpta, good_panels, field_data)

### Scale ground spectra to insolation curve

For each line, calculate the mean panel spectrum and determine the fit at the according mean Solar zenith angle.<BR>
Then determine individual fits for all ground data within the line and apply the ratio of the panel mean fit to<BR>
ground fit to the good_grounds dataframe.

In [ ]:
good_grounds = ScaleGrounds.scale_grounds(good_panels, good_grounds, slope, intercept, field_data)

### Define the K-factor

This reads a standard file with a response curve for the detector, given an ideally white surface.<BR>
Then "k_f" is defined for the K-factor.

In [ ]:
k_f = KFactor.k_factor(panel_dir, in_panel)

In [ ]:
k_f = k_f[k_f.index <910]

### Re-Make Spectral Dataframes

After re-scaling of both good_panels and good_grounds, recreate the good_panel_spec and<BR>
good_grounds_spec dataframes

In [ ]:
good_panel_spec = MakeSpecDF.make_spec_df(good_panels)
good_grounds_spec = MakeSpecDF.make_spec_df(good_grounds)

### Rename the first spectrum in ALL/GOOD panels to the correct name

Rather than just "radiance", it will be named something like radiance1-0<BR>
for the zeroth spectrum in the first line, for example.    

In [ ]:
SpecRename.spec_rename(good_panel_spec, good_grounds_spec, firstGoodLine, firstGoodPanelSpec, firstGoodGroundSpec, field_data)

In [ ]:
good_panel_spec = good_panel_spec.rename(columns={x:y for x,y in zip(good_panel_spec.columns,range(len(good_panel_spec.columns)))}).add_prefix('radiance1-')
good_grounds_spec = good_grounds_spec.rename(columns={x:y for x,y in zip(good_grounds_spec.columns,range(len(good_grounds_spec.columns)))}).add_prefix('radiance1-')


### Create dataframe with Reflectances

In [ ]:
all_refls = CreateReflectances.create_reflectances(good_panels, good_panel_spec, good_grounds_spec, k_f, field_data)

In [ ]:
all_refls.mean(axis=1).plot()

In [ ]:
blac = MakeSpecDF.make_spec_df(avgdata)

In [ ]:
all_grounds_trunc = all_grounds_spec.copy()
for i in all_grounds_trunc.columns:
    if all_grounds_trunc[i].iloc[0] > 0.025:
        all_grounds_trunc.drop(i, axis=1, inplace=True)

In [ ]:
all_grounds_trunc.mean(axis=0).plot()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12.5, 5.5))
plt.tight_layout(pad=5.5, w_pad=1.0, h_pad=1.0)
plt.scatter(cutavgdata[cutavgdata.Wavelength==350].date_saved, cutavgdata[cutavgdata.Wavelength==350].radiance)
#allASDdata[allASDdata.Wavelength==350].plot('date_saved', 'radiance', ax=axes)
#axes.set_ylim(0.0,0.005)
axes.set_xlim(datetime(2020,3,20,0,23), datetime(2020,3,20,0,59))

In [ ]:
plop = avgdata[np.logical_and(avgdata.date_saved > datetime(2020,3,20,0,27,53), avgdata.date_saved < datetime(2020,3,20,0,29))]

In [ ]:
plopmean = MakeSpecDF.make_spec_df(plop)

In [ ]:
plopmean.mean(axis=1).plot()

In [ ]:
ASDplop = allASDdata[np.logical_and(allASDdata.date_saved > datetime(2020,3,20,0,27,53), allASDdata.date_saved < datetime(2020,3,20,0,29))]

In [ ]:
ASDplop = ASDplop.rename(columns={'Spec_number': 'SpecNum'})

In [ ]:
ASDplopmean = MakeSpecDF.make_spec_df(ASDplop)

In [ ]:
ASDplopmean.mean(axis=1).plot()

In [ ]:
(plopmean.mean(axis=1)/ASDplopmean.mean(axis=1)).plot()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12.5, 5.5))
plt.tight_layout(pad=5.5, w_pad=1.0, h_pad=1.0)
plopmean.mean(axis=1).plot(ax=axes)
ASDplopmean.mean(axis=1).plot(ax=axes)
#axes.set_ylim(0.0,0.05)
axes.set_xlim(350,910)
plt.legend(['Flame','ASD'])
plt.xlabel('Wavelength (nm)')
plt.ylabel('Radiance (W m$^{-2}$ nm$^{-1}$ sr$^{-1}$)')
plt.title('Mullion 20MAR20 average of all ground spectra using GT08 calibration')

plt.savefig('plonk0.png')

In [ ]:
DarkCurrent = pd.read_csv('../CALIBRATION/FLAME_calibration_GA_ORIGINAL/Analysed/Analysed/DC_GT8.csv')

In [ ]:
DarkCurrent.set_index('wl', inplace=True)